# Import

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.evalModel import evalLogisticModels, getPredDist

# Load data

## Cell lines

In [3]:
trainRNA = pd.read_csv('../../data/processed/RNA_train_cancergenes.csv', index_col=0)
trainCellLines = list(trainRNA.index)

testRNA = pd.read_csv('../../data/processed/RNA_test_cancergenes.csv', index_col=0)
testCellLines = list(testRNA.index)

newRNA = pd.read_csv('../../data/processed/RNA_newcancer_cancergenes.csv', index_col=0)
newCellLines = list(newRNA.index)

## CDR

In [4]:
cdr = pd.read_csv('../../data/processed/drugCellLinePairsData.csv', index_col='DepMap_ID')
trainCDR = cdr.loc[trainCellLines, :].reset_index()
testCDR = cdr.loc[testCellLines, :].reset_index()
newCDR = cdr.loc[newCellLines, :].reset_index()

In [5]:
testTemp = testCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                             'name': 'drug',
                                                                                             'effective': 'true'})

newTemp = newCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                          'name': 'drug',
                                                                                          'effective': 'true'})

## Drugs

In [6]:
drugs = pd.read_csv('../../data/processed/drug_fingerprints.csv', index_col=0)

In [7]:
trainDrugs = drugs.loc[list(trainCDR.name.values), :].to_numpy()
testDrugs = drugs.loc[list(testCDR.name.values), :].to_numpy()
newDrugs = drugs.loc[list(newCDR.name.values), :].to_numpy()

trainRNA = trainRNA.loc[list(trainCDR.DepMap_ID.values), :].to_numpy()
testRNA = testRNA.loc[list(testCDR.DepMap_ID.values), :].to_numpy()
newRNA = newRNA.loc[list(newCDR.DepMap_ID.values), :].to_numpy()

## Format Pairs

In [8]:
trainData = [trainDrugs, trainRNA]
trainEff = trainCDR.effective.to_numpy()
testData = [testDrugs, testRNA]
newData = [newDrugs, newRNA]

In [9]:
del cdr, drugs, trainDrugs, testDrugs, newDrugs, trainRNA, testRNA, newRNA

# Model

In [10]:
drugPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/drugs/siameseV1/models/'
drugModel = 'DrugFewShot_Layers2_Hidden16_DO0-1_AFrelu_LR0-01_DR0-99_DS1000/'
drugPath = os.path.join(drugPath, drugModel)

In [11]:
evalLog = evalLogisticModels(trainData.copy(), trainEff, testData.copy(), newData.copy(), alt='xgb',
                            fusionPath=None, drugPath=drugPath, rnaPath=None)

In [12]:
evalLog.model.best_params_

{'learning_rate': 0.01, 'max_depth': 9, 'objective': 'binary:logistic'}

## Cell line performance

In [13]:
testDF, newDF, testWrong, newWrong, testCounts, newCounts = evalLog.evaluate(testDF=testTemp.copy(), 
                                                                             newDF=newTemp.copy())

Average Cell Line precision @ k on test set
	Precision@1: 0.8824
	Precision@2: 0.902
	Precision@3: 0.902
	Precision@4: 0.8775
	Precision@5: 0.8314
	Precision@10: 0.8205

Average Cell Line precision @ k on newcancer set
	Precision@1: 0.9538
	Precision@2: 0.9077
	Precision@3: 0.8872
	Precision@4: 0.8654
	Precision@5: 0.8462
	Precision@10: 0.8196


Test set:

	# of cell lines without effective drug among top-3 recs: 0
	# of unique drugs among top-3 predictions: 18

New cancer set
No true effective drugs identified in top 3 for ACH-000268 (top drug: sangivamycin)

	# of cell lines without effective drug among top-3 recs: 1
	# of unique drugs among top-3 predictions: 17


## Pred distribution

In [14]:
testDist = getPredDist(testDF)

Avg varaince of predictions for each drug: 0.00039999998989515007


In [15]:
newDist = getPredDist(newDF)

Avg varaince of predictions for each drug: 0.00039999998989515007


## Cancer performance

In [19]:
testCancer, newCancer = evalLog.getCancerPerformance(testDF, newDF)

In [20]:
print(round(testCancer.mean(), 4))
testCancer

p1    0.8797
p2    0.9091
p3    0.9134
p4    0.8879
p5    0.8373
dtype: float64


,p1,p2,p3,p4,p5
Liver Cancer,1.000000,1.000000,1.000000,1.000000,1.000000
Colon/Colorectal Cancer,1.000000,1.000000,1.000000,1.000000,0.950000
Bladder Cancer,1.000000,1.000000,1.000000,1.000000,0.933333
Ovarian Cancer,1.000000,1.000000,1.000000,0.937500,0.850000
Endometrial/Uterine Cancer,1.000000,1.000000,1.000000,0.916667,0.866667
Head and Neck Cancer,1.000000,0.833333,0.888889,0.833333,0.800000
Lung Cancer,0.923077,0.884615,0.871795,0.846154,0.830769
Skin Cancer,0.800000,0.900000,0.866667,0.850000,0.800000
Brain Cancer,0.750000,0.875000,0.833333,0.875000,0.800000
Pancreatic Cancer,0.750000,0.750000,0.833333,0.812500,0.750000


In [21]:
print(round(newCancer.mean(), 4))
newCancer

p1    0.9721
p2    0.9377
p3    0.9033
p4    0.8863
p5    0.8489
dtype: float64


,p1,p2,p3,p4,p5
Neuroblastoma,1.000000,1.000000,1.000000,1.000000,0.866667
Gallbladder Cancer,1.000000,1.000000,1.000000,1.000000,0.800000
Sarcoma,1.000000,1.000000,0.944444,0.916667,0.900000
Rhabdoid,1.000000,1.000000,0.916667,0.875000,0.850000
Prostate Cancer,1.000000,1.000000,0.833333,0.875000,0.900000
Gastric Cancer,1.000000,0.964286,0.952381,0.928571,0.900000
Bone Cancer,1.000000,0.944444,0.925926,0.861111,0.888889
Bile Duct Cancer,1.000000,0.800000,0.800000,0.750000,0.760000
Thyroid Cancer,0.875000,0.937500,0.916667,0.906250,0.900000
Kidney Cancer,0.846154,0.730769,0.743590,0.750000,0.723077
